In [88]:
import re
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.svm import SVC
from sklearn.svm import LinearSVC as LSVC
from sklearn.decomposition import TruncatedSVD as TSVD
from scipy.sparse import hstack
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

# SK-learn libraries for model selection 
from sklearn.model_selection import train_test_split

# json libraries to parse json file
import json
from pandas.io.json import json_normalize

import gensim
from gensim import utils


In [17]:
os.chdir("/Users/gurditchahal/W207/coursework/Final_Project_Random_Acts/EDA")
# read json file
train_json = json.load(open('train.json'))

# normalize data and put in a dataframe
train_json_df = json_normalize(train_json)

# read json file
test_json = json.load(open('test.json'))

# normalize data and put in a dataframe
test_json_df = json_normalize(test_json)

print("Train shape: ", train_json_df.shape)
print("Test shape: ", test_json_df.shape)

train_only_columns = set(train_json_df.columns.values)-set(test_json_df.columns.values)
print("Columns in Train but not Test:\n",train_only_columns)
test_only_columns = set(test_json_df.columns.values)-set(train_json_df.columns.values)
print("\nColumns in Test but not Train:",test_only_columns)
test_w_train_col = train_json_df[test_json_df.columns.values]

Train shape:  (4040, 32)
Test shape:  (1631, 17)
Columns in Train but not Test:
 {'request_text', 'number_of_downvotes_of_request_at_retrieval', 'requester_account_age_in_days_at_retrieval', 'requester_upvotes_plus_downvotes_at_retrieval', 'requester_number_of_posts_on_raop_at_retrieval', 'requester_number_of_comments_in_raop_at_retrieval', 'requester_received_pizza', 'request_number_of_comments_at_retrieval', 'post_was_edited', 'requester_user_flair', 'requester_number_of_comments_at_retrieval', 'number_of_upvotes_of_request_at_retrieval', 'requester_upvotes_minus_downvotes_at_retrieval', 'requester_days_since_first_post_on_raop_at_retrieval', 'requester_number_of_posts_at_retrieval'}

Columns in Test but not Train: set()


In [18]:
# 0 means the user doesn't receive pizza & 1 means the user receives pizza
train_labels = train_json_df.requester_received_pizza.astype(int).as_matrix()

# split the training data into training data and dev data 
train_data, dev_data, train_labels, dev_labels = \
            train_test_split(test_w_train_col, train_labels, test_size=0.2, random_state=12)
    

In [19]:
train_text=train_data['request_text_edit_aware'] +' '+train_data['request_title']
dev_text=dev_data['request_text_edit_aware'] +' '+dev_data['request_title']

In [20]:
# Narratives per Standford paper
money = ["money", "now", "broke", "week", "until", "time",
          "last", "day", "when", "today", "tonight", "paid", "next",
          "first", "night", "after", "tomorrow", "month", "while",
          "account", "before", "long", "Friday", "rent", "buy",
          "bank", "still", "bills", "ago", "cash", "due",
          "soon", "past", "never", "paycheck", "check", "spent",
          "years", "poor", "till", "yesterday", "morning", "dollars",
          "financial", "hour", "bill", "evening", "credit",
          "budget", "loan", "bucks", "deposit", "dollar", "current",
          "payed"]
job =["work", "job", "paycheck", "unemployment", "interview",
          "fired", "employment", "hired", "hire"]
student = ["college", "student", "school", "roommate",
          "studying", "university", "finals", "semester",
          "class", "study", "project", "dorm", "tuition"]
family =["family", "mom", "wife", "parents", "mother", "husband",
           "dad", "son", "daughter", "father", "parent",
           "mum"]
craving = ["friend", "girlfriend", "craving", "birthday",
          "boyfriend", "celebrate", "party", "game", "games",
          "movie", "date", "drunk", "beer", "celebrating", "invited",
          "drinks", "crave", "wasted", "invite"]

narratives = [money, job, student, family, craving]




In [21]:
import string

def pre_proccess(s):
    s = re.sub("[^\w']|_", " ", s) 
    s=s.translate(str.maketrans(' ',' ',string.punctuation))#strip punctuation before looking
    s=s.lower()
    return s


In [31]:
def construct_ft_mat(train_data):
    feat_mat=pd.DataFrame()
    feat_mat['hour_request']=pd.to_datetime(train_data['unix_timestamp_of_request_utc'],unit = 's').dt.hour
    feat_mat['hour_request']=pd.to_datetime(train_data['unix_timestamp_of_request_utc'],unit = 's').dt.day
    feat_mat['hour_request']=pd.to_datetime(train_data['unix_timestamp_of_request_utc'],unit = 's').dt.month
    #feat_mat['month_request']=pd.to_datetime(train_data['unix_timestamp_of_request'],unit = 's').dt.month
    feat_mat['first_post']=np.log(train_data['requester_days_since_first_post_on_raop_at_request']+1)
    feat_mat['upvotes_minus_downvotes']=train_data['requester_upvotes_minus_downvotes_at_request']
    feat_mat['upvotes_plus_downvotes_at_request']=np.log(train_data['requester_upvotes_plus_downvotes_at_request']+1)
    upvotes=train_data.apply(lambda row: (row['requester_upvotes_plus_downvotes_at_request'] + row['requester_upvotes_minus_downvotes_at_request'])/2,axis=1)
    downvotes=train_data.apply(lambda row: (row['requester_upvotes_plus_downvotes_at_request']- row['requester_upvotes_minus_downvotes_at_request'])/2,axis=1)
    feat_mat['upvotes']=upvotes
    feat_mat['vote_ratio']=upvotes/(upvotes+downvotes+1)
    feat_mat['req_age']=np.log(train_data['requester_account_age_in_days_at_request']+1)
    feat_mat['num_subs']=np.log(train_data['requester_number_of_subreddits_at_request']+1)
    feat_mat['num_posts']=np.log(train_data['requester_number_of_posts_at_request']+1)
    feat_mat['pizza_activity']=np.log(train_data['requester_number_of_posts_on_raop_at_request']+1)
    feat_mat['len_request']=np.log(train_data['request_text_edit_aware'].apply(len)+1)
    feat_mat['len_title']=np.log(train_data['request_title'].apply(len)+1)
    #feat_mat['len_name']=train_data['requester_username'].apply(len)
    feat_mat['pizza_comments']=np.log(train_data['requester_number_of_comments_in_raop_at_request']+1)
    feat_mat['hyperlink'] = train_data['request_text_edit_aware'].apply(lambda x: 1 if re.search("http|www", x) else 0)
    # reciprocity indicator
    feat_mat['reciprocity'] = train_data['request_text_edit_aware'].apply(lambda x:1 if re.search("repay|pay.+back|pay.+forward|return.+favor", x) 
                                               else 0)
    feat_mat['politeness'] = train_data['request_text_edit_aware'].apply(lambda x: 1 if re.search("thank|appreciate|advance", x) else 0)
    
    return feat_mat

In [32]:
def find_narr(narr,s):
    ct=0
    for word in narr:
        ct+=s.split().count(word)
    return ct/len(s.split())

In [33]:
feat_mat=construct_ft_mat(train_data)

dev_mat=construct_ft_mat(dev_data)


In [34]:
train_text_clean=train_text.apply(lambda s:pre_proccess(s))
dev_text_clean=dev_text.apply(lambda s: pre_proccess(s))     
for n in narratives:
    feat_mat[n[0]]=train_text_clean.apply(lambda s: find_narr(n,s))
    dev_mat[n[0]]=dev_text_clean.apply(lambda s: find_narr(n,s))
    

t_mat=feat_mat.as_matrix()

d_mat=dev_mat.as_matrix()
   

In [57]:
vectorizer = CountVectorizer(min_df=5,ngram_range=(1,2), preprocessor=pre_proccess,stop_words='english') 
train_bag_of_words = vectorizer.fit_transform(train_text)
dev_bag_of_words = vectorizer.transform(dev_text)

In [36]:
'''rf=RF()
#from sklearn.preprocessing import StandardScaler
#S=StandardScaler()
#clf=LogisticRegression()
r_parameters = {'n_estimators':[64,100,128],'criterion':['gini','entropy'],'random_state':[42],'max_depth':[None,1,3,5,7],'max_features':[None,'auto'],'min_samples_leaf':[1,3,5]}
clf=GridSearchCV(rf, r_parameters,scoring='roc_auc')
clf.fit(t_mat,train_labels)
preds=clf.predict(d_mat)
roc_auc_score(dev_labels, preds, average='micro')'''

"rf=RF()\n#from sklearn.preprocessing import StandardScaler\n#S=StandardScaler()\n#clf=LogisticRegression()\nr_parameters = {'n_estimators':[64,100,128],'criterion':['gini','entropy'],'random_state':[42],'max_depth':[None,1,3,5,7],'max_features':[None,'auto'],'min_samples_leaf':[1,3,5]}\nclf=GridSearchCV(rf, r_parameters,scoring='roc_auc')\nclf.fit(t_mat,train_labels)\npreds=clf.predict(d_mat)\nroc_auc_score(dev_labels, preds, average='micro')"

In [93]:
lsvc = LSVC(C=0.024, penalty="l1", dual=False).fit(train_bag_of_words,train_labels)
model = SelectFromModel(lsvc, prefit=True)
S=StandardScaler(with_mean=False)
X_new = model.transform(S.fit_transform(train_bag_of_words))
print(X_new.shape)

d_new=model.transform(S.transform(dev_bag_of_words))

f_new=hstack([X_new,t_mat])
dev_new=hstack([d_new,d_mat])

(3232, 60)


/Users/gurditchahal/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [94]:
nb=BernoulliNB()
#nb=MultinomialNB()
n_parameters = {'alpha': np.linspace(0.01, 10, 100),'binarize':np.linspace(0.0, 1, 10)}
clf2 = GridSearchCV(nb, n_parameters,scoring='roc_auc',cv=5)
clf2.fit(f_new,train_labels)
preds=clf2.predict(dev_new)
print(clf2.best_params_)
print(roc_auc_score(dev_labels, preds, average='micro'))


{'binarize': 1.0, 'alpha': 10.0}
0.5942763157894737


In [95]:
from sklearn.ensemble import GradientBoostingClassifier
GBC = GradientBoostingClassifier(
    n_estimators=500, 
    learning_rate=0.01, 
    max_depth=4, 
    random_state=123)

GBC.fit(f_new,train_labels)
preds=GBC.predict(dev_new)
roc_auc_score(dev_labels, preds, average='micro')

0.5542434210526316

In [90]:
lsvm=LSVC(penalty='l2',dual=True)
l_param={'C':np.linspace(0.001,1,100),'class_weight':['balanced',None]}
csvm=GridSearchCV(lsvm,l_param,scoring='roc_auc',cv=5)
S=StandardScaler(with_mean=False)
csvm.fit(S.fit_transform(f_new),train_labels)
preds=csvm.predict(S.transform(dev_new))
roc_auc_score(dev_labels, preds, average='micro')

0.5580592105263158